# Project 1: Projectile Motion 3D

## Tips

  * Python tutorials:
     * PythonMinimum
     * [A short introduction](https://realpython.com/python-first-steps/)
     * [A more complete introduction](https://www.w3schools.com/python/default.asp)
  * Use __esc r__ to disable a cell
  * Use __esc y__ to reactivate it
  * Use __esc m__ to go to markdown mode. **Markdown** is the typesetting language used in jupyter notebooks.
  * In a markdown cell, double tap the mouse or glide pad (on your laptop) to go to edit mode. 
  * Shift + return to execute a cell (including markdown cells).
  * If the equations don't typeset, try double tapping the cell again, and re-execute it.


## Goal

This notebook provides a gentle introduction to `vpython`. 

## Numerical Solution of Newton's Second Law

### Newton's second law
Newton's second law of motion for a particle of mass $m$ is

$$\vec{F} = m \vec{a},$$ 

where, by definition, the acceleration is

$$\vec{a} = \frac{d\vec{v}}{dt}, $$

and $\vec{F}$ is the sum of all forces acting on the particle. (Note: The second law applies to every particle that comprises an extended object.) The second law can be written as two first order **ordinary differential equations** (ODE),

\begin{align}
    \frac{d\vec{r}}{dt} & = \vec{v}, \\
    \frac{d\vec{v}}{dt} & = \frac{1}{m} \vec{F} .
\end{align}

These equations can be solved approximately using the formulae

\begin{align}
    \vec{r}(t + h) & = \vec{r}(t) + \vec{v}(t) \, h  + \frac{1}{2} \frac{\vec{F}(t)}{m}  \, h^2 + {\cal O}(h^3),\\
    \vec{v}(t + h) & = \vec{v}(t) + \frac{\vec{F}(t)}{m}  \, h + {\cal O}(h^2) .
\end{align}

The symbol ${\cal O}(h^3)$ represents all terms proportional to $h^3$ and to higher powers of $h$. Likewise for ${\cal O}(h^2)$. We'll choose $h$ small enough so that these terms can be neglected. 

### Import modules 
Make Python modules (that is, collections of programs) available to this notebook.


In [1]:
import os, sys
import numpy as np
import matplotlib as mp
import matplotlib.pyplot as plt
import sympy as sm
import vpython as vp

sm.init_printing()        # activate "pretty printing" of symbolic expressions
%matplotlib inline

# update fonts
FONTSIZE = 14
font = {'family' : 'sans-serif',
        'weight' : 'normal',
        'size'   : FONTSIZE}
mp.rc('font', **font)

# use latex if available on system, otherwise set usetex=False
mp.rc('text', usetex=True)

# use JavaScript for rendering animations
mp.rc('animation', html='jshtml')

# set a seed to ensure reproducibility 
# on a given machine
seed = 314159
rnd  = np.random.RandomState(seed)

<IPython.core.display.Javascript object>

## Simulating a projectile

  1. **Step 1**: We write a function that implements the vector equations.
  \begin{align}
    \vec{r}(t + h) & = \vec{r}(t) + \vec{v}(t) \, h  + \frac{1}{2} \frac{\vec{F}(t)}{m}  \, h^2 + {\cal O}(h^3),\\
    \vec{v}(t + h) & = \vec{v}(t) + \frac{\vec{F}(t)}{m}  \, h + {\cal O}(h^2) .
\end{align}
  1. **Step 2**: We write a function that implements the total force on the ball.
  1. **Step 3**: We write a function that computes the trajectory.
  1. **Step 3**: Plot the trajectory.

### Constants

In [2]:
# simulation constants
M  = 0.25               # mass of ball (kg)
MU = 0.0                # friction constant N/m/s (N: Newton, unit of force)
g  = 9.81               # acceleration due to gravity (m/s^2)
H  = 0.01               # time step (seconds) 
R0 = np.array((0,1,0))  # initial position of ball
V0 = np.array((4,10,0))
class Bag:              # a very simple class (see PythonMinimum)
    pass

bag    = Bag()
bag.h  = H    # time step (seconds)
bag.hh = H**2
bag.g  = np.array((0, -g, 0)) # acceleration due to gravity
bag.mu = MU   # friction constant (N/m/s)
bag.m  = M    # mass of ball (kg)

## Step 1: Propagator
We start by writing a function that takes the current position $\vec{r}(t)$ and velocity $\vec{v}(t)$ of the (center of the) ball, which is treated as a particle, the sum of the forces acting on the ball (often referred to as the **net force**), $\vec{F}(t)$, and returns the updated position and velocity at timestamp $t + h$ of the particle. 

\begin{align}
    \vec{r}(t + h) & = \vec{r}(t) + \vec{v}(t) \, h  + \frac{1}{2} \frac{\vec{F}(t)}{m}  \, h^2 + {\cal O}(h^3),\\
    \vec{v}(t + h) & = \vec{v}(t) + \frac{\vec{F}(t)}{m}  \, h + {\cal O}(h^2) .
\end{align}

In [3]:
def propagate(r, v, F, bag):    
    h, hh, m = bag.h, bag.hh, bag.m
    
    Fm   = F/m
    rnew = r + v*h + Fm*hh/2
    vnew = v + Fm*h
    
    return rnew, vnew   # return new position and  velocity

## Step 2: Force

The function below returns the total force on the particle. We consider two forces, the force of gravity,

\begin{align}
    \vec{F}_g(t) & = m \vec{g} .
\end{align}

and a simple friction force,

\begin{align}
    \vec{F}_{\mu}(t) & = - \mu  \vec{v} ,
\end{align}

where $\mu$ is a constant.

In [4]:
def force(r, v, bag):
    m = bag.m
    g = bag.g
    mu= bag.mu
    F = m * g - mu * v
    return F

## Step 3: Compute Trajectory
We'll stop the calculations when the projectile reaches the ground.

In [5]:
def compute_trajectory(r0, v0, bag, max_steps=100000):
    '''
    
    r0     initial position
    v0     initial velocity
    '''
    
    # set initial state (r and v are lists of vectors)
    r  = [r0]
    v  = [v0]

    # loop over time steps and stop when the ball hits the ground.
    # notice the indentation with respect to the for i range(max_steps) command.
    # this is how Python knows that the instructions that follow are within 
    # the loop (or if statement etc.)
    for i in range(max_steps):
            
        # compute total force on ball
        F = force(r[i], v[i], bag)

        # compute next state of ball (next position and velocity at time t + h)
        rnext, vnext = propagate(r[i], v[i], F, bag)

        # check if we've reached the ground
        x, y, z = rnext # rnext is a vector in 3D space, so it has 3 components
        if y <= 0:
            break # break out of the loop

        # cache (that is save) the next state in the lists r and v
        r.append(rnext)
        v.append(vnext)
    
    return r, v # return the two lists

In [6]:
bag.r, bag.v = compute_trajectory(R0, V0, bag)

## Step 4: Plot Results
We are going to use the 3D animation module `vpython` to visualize the calculations above by animating the motion of the ball.

For the vector algebra, we'll use the very convenient `vpython` vector class. You can see the attributes and functions that are available for this class using the Python `help` function. 

In [7]:
help(vp.vector)

Help on class vector in module vpython.cyvector:

class vector(builtins.object)
 |  Methods defined here:
 |  
 |  __add__(self, value, /)
 |      Return self+value.
 |  
 |  __eq__(self, value, /)
 |      Return self==value.
 |  
 |  __ge__(self, value, /)
 |      Return self>=value.
 |  
 |  __gt__(self, value, /)
 |      Return self>value.
 |  
 |  __init__(self, /, *args, **kwargs)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __le__(self, value, /)
 |      Return self<=value.
 |  
 |  __lt__(self, value, /)
 |      Return self<value.
 |  
 |  __mul__(self, value, /)
 |      Return self*value.
 |  
 |  __ne__(self, value, /)
 |      Return self!=value.
 |  
 |  __neg__(self, /)
 |      -self
 |  
 |  __pos__(self, /)
 |      +self
 |  
 |  __radd__(self, value, /)
 |      Return value+self.
 |  
 |  __reduce__ = __reduce_cython__(...)
 |  
 |  __repr__(self, /)
 |      Return repr(self).
 |  
 |  __rmul__(self, value, /)
 |      Return value*self.


## Graphics Constants

In [8]:
TRAIL       = True       # add a trail to ball
TRAIL_COUNT = 300        # maximum trail length attached to ball

# in vpython, RGB colors must be defined using the vpython vector class
SKYBLUE   = vp.vector(0.62,0.57,0.98)
LAWNGREEN = vp.vector(0.5,0.9,0.5)
GRAY      = vp.vector(0.70,0.70, 0.70)

WIDTH  = 400 # viewport width in pixels
HEIGHT = 400 # viewport height in pixels

ORIGIN = vp.vector(0,0,0)
I      = vp.vector(1,0,0) # unit vector in x direction
J      = vp.vector(0,1,0) # unit vector in y direction
K      = vp.vector(0,0,1) # unit vector in z direction
CAMERA = vp.vector(-1, -0.5, -0.8) # direction in which camera points

# simulation constants
SIZE = 10
HW   = SIZE   # half width of "ground"
SW   = HW/100 # shaftwidth of coordinate arrows

### Build scene elements
  * `create_canvas`: create the canvas on which the scene will be drawn. We view the scene through a viewport of size `WIDTH` and `HEIGHT` in pixels.
  * `draw_axes`: draw right-handed Cartesian coordinate axes and a plane in the $y-z$ plane.
  * `build_scene`: build the scene objects

In [9]:
def create_canvas(bag):
    bag.scene = vp.canvas()
    bag.scene.caption= 'A non-bouncing ball'
    bag.scene.width = WIDTH
    bag.scene.height= HEIGHT
    bag.scene.background=SKYBLUE
    bag.scene.userzoom  = False  # user can't zoom
    bag.scene.range  = SIZE      # window size in world coordinates
    bag.scene.up     = J         # direction of vertical
    bag.scene.forward= CAMERA    # direction in which camera looks
    
def draw_axes(w):
    sw = SIZE/100/2
    aw = 1.1*SIZE/2
    
    # draw ground
    a = vp.vertex( pos= w*I+w*K, color=LAWNGREEN)
    b = vp.vertex( pos= w*I-w*K, color=LAWNGREEN)
    c = vp.vertex( pos=-w*I-w*K, color=vp.color.red)
    d = vp.vertex( pos=-w*I+w*K, color=LAWNGREEN)
    xzplane = vp.quad(vs=[a, b, c, d])
    
    # draw Cartesian axes 
    xaxis = vp.arrow(pos=ORIGIN, axis=w*I, shaftwidth=sw, color=GRAY)
    xlabel= vp.label(pos=aw*I, text='x', box=False) 
    
    yaxis = vp.arrow(pos=ORIGIN, axis=w*J, shaftwidth=sw, color=GRAY)
    ylabel= vp.label(pos=aw*J, text='y', box=False) 
    
    zaxis = vp.arrow(pos=ORIGIN, axis=w*K, shaftwidth=sw, color=GRAY)
    zlabel= vp.label(pos=aw*K, text='z', box=False) 
    
    return xzplane, xaxis, yaxis, zaxis, xlabel, ylabel, zlabel

def build_scene(bag):
    
    scene = bag.scene
    
    # cache all widgets in bag to prevent them from getting deleted
    # inadvertently and to make them accessible to the update function
    
    bag.xyz = draw_axes(HW)
    
    # draw a wall
    bag.wall01 = vp.box(pos=vp.vector(-9, 5, 0),
                        up=J,        # direction of height of box
                        axis=-I,     # direction of length of box
                        length=0.2,
                        height=10,
                        width=20,
                        color=LAWNGREEN,
                        opacity=0.1)
    
    x, y, z = bag.r[0]
    bag.ball = vp.sphere(color=vp.color.red,
                         radius=0.4,
                         pos=vp.vector(x, y, z),
                         make_trail=TRAIL,
                         trail_radius=SW/2,
                         retain=TRAIL_COUNT) 
    
    # create controls
    bag.b_start = vp.button(text="Start",
                            background=vp.color.green,
                            pos=scene.title_anchor, 
                            bind=start)
    
    bag.b_stop  = vp.button(text="Stop",
                            background=vp.color.red,
                            pos=scene.title_anchor, 
                            bind=stop)
        
#     # min-speed:  1 m/s
#     # max-speed: 30 m/s
#     # default:   10 m/s
#     bag.s_speed = vp.slider(min=1.0, max=30, value=10, length=300, 
#                             pos=scene.title_anchor,
#                             bind=speed)
    
#     bag.stext   = vp.wtext(text=f'{bag.s_speed.value:5.2f} m/s', 
#                            pos=scene.title_anchor)
    
    bag.time   = vp.wtext(text=f'\ttime: {0:8.2f} s', 
                          pos=scene.title_anchor)
    
    bag.active = True       # event loop
    bag.update = False      # updating of scene initially turned off
    bag.frame  = 0          # initial frame number
    bag.nframes= len(bag.r) # number of frames (time steps)

In [10]:
def update(bag):
    # update position and velocity of ball unless
    # we've exhausted the number of frames in which
    # case just return
    bag.update = bag.frame < bag.nframes
    if not bag.update:
        return
    
    x, y, z = bag.r[bag.frame]        
    bag.ball.pos = vp.vector(x, y, z)
    
    # update frame counter
    bag.frame += 1
    bag.time.text = f'\ttime: {bag.h*bag.frame:8.2f} s'
    
def run(bag):
    
    while bag.active:
        if bag.update:
            update(bag) 
        vp.rate(100)
        
    print('animation stopped')
    
# controls
def start(b):
    global bag
    bag.update = True

def stop(b):
    global bag
    bag.active = False

def speed(s):
    global bag
    bag.stext.text = f'{bag.s_speed.value:5.2f} m/s'

In [11]:
create_canvas(bag)
build_scene(bag)
run(bag)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

animation stopped
